### **Transform Team Dimension**

### Configuration

In [1]:
%run /utils/general_functions

In [2]:
create_mounts()

### Load Data

In [3]:
job_id = mssparkutils.env.getJobId()

player_df = spark.read.format('delta').load(f'synfs:/{job_id}/mnt/silver/dim_player')

### Transformation

In [ ]:
from pyspark.sql.functions import when, col
players_final_df = player_df \
    .withColumn('first_name', when(col('first_name').isNull(), 'no_name').otherwise(col('first_name'))) \
    .withColumn('last_name', when(col('last_name').isNull(), 'no_name').otherwise(col('last_name'))) \
    .withColumn('position', when(col('position').isNull(), 'N/a').otherwise(col('position'))) \
    .withColumn('height_feet', when(col('height_feet').isNull(), 0).otherwise(col('height_feet'))) \
    .withColumn('height_inches', when(col('height_inches').isNull(), 0).otherwise(col('height_inches'))) \
    .withColumn('weight_pounds', when(col('weight_pounds').isNull(), 0).otherwise(col('weight_pounds')))

### Merge Data

In [4]:
container = 'gold'
database = 'prize_picks_gold'
table = 'dim_player'
file_format = 'delta'
merge_condition = 'tgt.player_key = src.player_key'

merge_data(players_final_df, container, database, table, file_format, merge_condition=merge_condition )

In [ ]:
%%sql
DROP TABLE IF EXISTS prize_picks_silver.dim_player;